# Multicriterio


In [2]:
import numpy as np
import pandas as pd

xls=pd.ExcelFile(r"C:\Users\marti\OneDrive\Documentos\personal\Universidad\Septimo_semestre\Opti_multiobjective\Multicriterio\imdbmoviesdata.xlsx")
datos=pd.read_excel(xls,"tmdb_movies_data")
tabla_AHP=pd.read_excel(xls,"AHP")
tabla_AHP_gen=pd.read_excel(xls,"AHP_genero")
tabla_AHP_time=pd.read_excel(xls,"AHP_duracion")


#[65,90,120,180,340]


dtype('float64')

In [5]:
#Implementación con más columnas categóricas
cols=datos.columns
num_cols=datos._get_numeric_data().columns
categ_cols=list(set(cols)-{"Nombre"}-set(num_cols))
tabla_AHP_gen.columns[0]

'Género'

In [25]:
def calc_ponder(matr:np.array,tol:float):
    """Recibe:
    matr: matriz nxn de comparación AHP dos a dos recíproca.
    tol: tolerancia de variación entre vectores suma normalizados.
    Retorna:
    vect_pond: vector ponderación. Eleva error si no se cumple consistencia"""
    n,_=matr.shape
    epsilon=np.array([tol+1])
    i=2
    prev_v_norm=matr.sum(axis=1)/matr.sum()
    while (epsilon>=tol).any():
        if i>6:
            raise Exception("Matriz no consistente por iteraciones")
        A_n=np.linalg.matrix_power(matr,i)#Eleva error si no es cuadrada
        v_norm=A_n.sum(axis=1)/A_n.sum() #vector de suma de filas normalizado
        epsilon=max(v_norm-prev_v_norm)
        prev_v_norm=v_norm
    #Verificación consistencia
    v1=np.matmul(matr,v_norm)
    v1_perc=np.divide(v1,v_norm)
    print(v1,v1_perc)
    lambda_max=np.mean(v1_perc)
    print(lambda_max)
    indice_RI={1:0,3:0.58,4:0.89,5:1.11,6:1.24,7:1.32,8:1.4,9:1.5,10:1.49}
    RI=indice_RI[n]
    CI=(lambda_max-n)/(n-1)
    print(CI)
    CR=CI/RI
    print(CR)
    if CR>0.1:
        raise Exception("Matriz no consistente por prueba")
    return v_norm

def categ_AHP(tabla_AHP:pd.DataFrame):
    """Recibe:
    tabla_AHP: tabla comparativa sobre los posibles valores categóricos de la variable
    Retorna:
    Diccionario con valores:ponderaciónAHP"""
    criterio=tabla_AHP.columns[0] #La tabla debe tener como primer elemento del encabezado el nombre del criterio corespondiente
    
    
    
def AHP(norm_data,vect_pond):
    pass

#datos["Género"].unique()
#matr=np.array([[1,3,5,3,5],[0.33,1,2,1,3],[0.2,0.5,1,1,2],[0.33,1,1,1,4],[0.2,0.33,0.5,0.25,1]])

#matr=np.array([[1,3,0.25,5,6],[1/3,1,0.5,2,4],[4,2,1,6,9],[0.2,0.5,1/6,1,3],[1/6,0.25,1/9,1/3,1]])
matr=np.array([[1,0.5,1,3],[2,1,8,9],[1,0.125,1,2],[1/3,1/9,1/2,1]])
calc_ponder(matr,0.005)

[0.81207271 2.54200011 0.51730351 0.25650538] [4.17975996 4.08699727 4.23287077 4.16881998]
4.167111996093883
0.055703998697961055
0.06258876258197871


array([0.19428693, 0.62197255, 0.12221103, 0.06152949])

In [4]:
b=np.array([3,1,3])
c=np.array([6,10,6])
np.divide(c,b)

array([ 2., 10.,  2.])

In [5]:
tabla_AHP_gen.head()

,Unnamed: 0,Action,Adventure,Animation,Comedy,Crime,Drama,Familiy,Fantasy,Horror,Music,Mystery,Romance,Science Fiction,Thriller,War,Western
0,Action,1.000000,2.000000,1.500000,3.0,4.0,1.5,4.0,3.0,5.0,2.0,3.0,1.5,2.0,1.5,1.5,2.0
1,Adventure,0.500000,1.000000,0.900000,1.5,2.0,1.0,2.0,1.0,3.0,1.0,2.0,0.5,1.0,1.5,0.8,1.0
2,Animation,0.666667,1.111111,1.000000,2.0,3.0,1.0,3.0,2.0,4.0,1.5,2.0,1.0,1.5,1.0,1.0,1.5
3,Comedy,0.333333,0.666667,0.500000,1.0,2.0,0.5,3.0,1.0,3.0,0.5,1.0,0.5,0.8,0.5,0.5,0.8
4,Crime,0.250000,0.500000,0.333333,0.5,1.0,0.3,2.0,0.8,2.0,0.3,0.5,0.4,0.5,0.3,0.3,0.5
